In [ ]:
pip install transformers pandas torch scikit-learn


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
print(df.columns)

Index(['Law/Requirement Name', 'Description', 'Relevant Authority',
       'Compliance Steps', 'Required Documents',
       'Penalties for Non-Compliance', 'Frequency of Compliance', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('/content/legal_dataset.csv')

# Assuming columns: 'question', 'context', 'answer', and other features
df = df[['Act', 'Document Requirements', 'Conditions']]

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(df):
    return tokenizer(
        list(df['Act'].values),
        list(df['Conditions'].values),
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch

class LegalAdvisoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, answers):
        self.encodings = encodings
        self.answers = answers

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.answers[idx])
        return item

    def __len__(self):
        return len(self.answers)

train_dataset = LegalAdvisoryDataset(train_encodings, train_df['Document Requirements'].values)
val_dataset = LegalAdvisoryDataset(val_encodings, val_df['Document Requirements'].values)


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['Document Requirements'].unique()))


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [ ]:
!pip install transformers[torch]

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
pip install accelerate -U


In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, answers):
        self.encodings = encodings
        self.answers = answers

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        # Ensure labels are converted to numeric type
        item['labels'] = torch.tensor(int(self.answers[idx]))  # Assuming answers are integers

        return item


In [ ]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification # Import the necessary model class

# Assuming you want to use a pre-trained model from Hugging Face Model Hub
model_name = "bert-base-uncased"  # Example model name
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) # Replace num_labels with the number of classes in your classification task

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Create your training dataset
# Replace these with your actual data and preprocessing steps
encodings = {'input_ids': [[1, 2, 3], [4, 5, 6]], 'attention_mask': [[1, 1, 1], [1, 1, 1]]}
answers = [0, 1]
train_dataset = CustomDataset(encodings, answers)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertForSequenceClassification, BertTokenizer, get_linear_schedule_with_warmup
import numpy as np

# Custom Dataset class (assuming it's defined as described earlier)
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Assuming you have tokenized and encoded your data already
train_encodings = {'input_ids': np.random.randint(0, 100, (100, 50)), 'attention_mask': np.random.randint(0, 2, (100, 50))}
train_labels = np.random.randint(0, 2, 100)
val_encodings = {'input_ids': np.random.randint(0, 100, (20, 50)), 'attention_mask': np.random.randint(0, 2, (20, 50))}
val_labels = np.random.randint(0, 2, 20)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# Tokenizer and Model initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Optional: Gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss}')

    # Validation
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(val_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Training Loss: 0.7093366384506226
Epoch 1/3, Validation Loss: 0.7288681268692017
Epoch 2/3, Training Loss: 0.7087071079474229
Epoch 2/3, Validation Loss: 0.7306572596232096
Epoch 3/3, Training Loss: 0.6811113082445585
Epoch 3/3, Validation Loss: 0.7570070028305054


In [ ]:
# Assuming previous imports and setup are already executed

# Training loop (continued)
for epoch in range(num_epochs):
    # Training
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Optional: Gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss}')

    # Validation
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass (validation)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(val_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss}')

# Save the trained model and tokenizer
output_dir = './legal_advisor_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Epoch 1/3, Training Loss: 0.708636265534621
Epoch 1/3, Validation Loss: 0.7512351274490356
Epoch 2/3, Training Loss: 0.6837635682179377
Epoch 2/3, Validation Loss: 0.7646560271581014
Epoch 3/3, Training Loss: 0.6987782945999732
Epoch 3/3, Validation Loss: 0.7969677448272705


('./legal_advisor_model/tokenizer_config.json',
 './legal_advisor_model/special_tokens_map.json',
 './legal_advisor_model/vocab.txt',
 './legal_advisor_model/added_tokens.json')

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Directory where the model and tokenizer are saved
output_dir = './legal_advisor_model'

# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Ensure the model is in evaluation mode
model.eval()

# Function to predict category for a given text
def predict_category(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

# Example usage
query = "What are the steps to register a trademark?"
predicted_class = predict_category(query)
print(f"Predicted class for '{query}': {predicted_class}")


Predicted class for 'What are the steps to register a trademark?': 0


In [ ]:
!pip install scikit-learn pandas transformers torch

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader

# Assuming you have adjusted the dataset and column names accordingly
df = pd.read_csv('/content/legal_dataset.csv')

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# ... rest of your code ...

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(df):
    return tokenizer(
        list(df['Act'].values),
        list(df['Conditions'].values),
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)

class LegalAdvisoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, answers):
        self.encodings = encodings
        # Map string labels to numerical labels
        unique_answers = list(set(answers))
        self.label_map = {label: i for i, label in enumerate(unique_answers)}
        self.answers = torch.tensor([self.label_map[ans] for ans in answers]) # Convert answers to numerical data

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.answers[idx] # No need to create a tensor again
        return item

    def __len__(self):
        return len(self.answers)

train_dataset = LegalAdvisoryDataset(train_encodings, train_df['Document Requirements'].values)
val_dataset = LegalAdvisoryDataset(val_encodings, val_df['Document Requirements'].values)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['Relevant Authority'].unique()))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Adjust the optimizer and scheduler parameters as needed
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Optional: Gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-14-889b03bd5241>:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
!pip install scikit-learn
import pandas as pd
from sklearn.model_selection import train_test_split # Import the missing function
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup # Import BertTokenizer here
import torch
from torch.utils.data import DataLoader
# Assuming you have adjusted the dataset and column names accordingly
df = pd.read_csv('/content/legal_dataset.csv')

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# ... rest of your code ...

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(df):
    return tokenizer(
        list(df['Act'].values),
        list(df['Conditions'].values),
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)

class LegalAdvisoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, answers):
        self.encodings = encodings
        # Map string labels to numerical labels
        unique_answers = list(set(answers))
        self.label_map = {label: i for i, label in enumerate(unique_answers)}
        self.answers = torch.tensor([self.label_map[ans] for ans in answers]) # Convert answers to numerical data

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.answers[idx] # No need to create a tensor again
        return item

    def __len__(self):
        return len(self.answers)

train_dataset = LegalAdvisoryDataset(train_encodings, train_df['Document Requirements'].values)
val_dataset = LegalAdvisoryDataset(val_encodings, val_df['Document Requirements'].values)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['Document Requirements'].unique()))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Adjust the optimizer and scheduler parameters as needed
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Optional: Gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss}')

    # Validation
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(val_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss}')

# Save the trained model and tokenizer
output_dir = './legal_advisor_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Function to predict category for a given text
def predict_category(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

# Modify the category mapping based on your dataset
# Modify the category mapping to use the label_map
categories = list(train_dataset.label_map.keys())

# Function for user interaction
def chatbot():
    print("Legal Advisory Chatbot is ready. Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Legal Advisory Chatbot: Goodbye!")
            break
        else:
            predicted_class = predict_category(user_input)
            predicted_category = categories[predicted_class]
            print(f"Legal Advisory Chatbot: Predicted category for '{user_input}': {predicted_category}")

# Start the chatbot
chatbot()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-21-2bd7702ee434>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/3, Training Loss: 3.216620087623596
Epoch 1/3, Validation Loss: 3.286382484436035
Epoch 2/3, Training Loss: 3.1764806628227236
Epoch 2/3, Validation Loss: 3.2789151668548584
Epoch 3/3, Training Loss: 3.0203555107116697
Epoch 3/3, Validation Loss: 3.293852186203003
Legal Advisory Chatbot is ready. Type 'exit' to end the chat.
You: gst
Legal Advisory Chatbot: Predicted category for 'gst': Evidence of damage, Complaint
You: tax
Legal Advisory Chatbot: Predicted category for 'tax': Legal purpose, Consideration proof
You: patent
Legal Advisory Chatbot: Predicted category for 'patent': Legal purpose, Consideration proof
You: registration
Legal Advisory Chatbot: Predicted category for 'registration': Proof of office address
You: trademark
Legal Advisory Chatbot: Predicted category for 'trademark': Legal purpose, Consideration proof
You: exit
Legal Advisory Chatbot: Goodbye!
